In [1]:
import pandas as pd
import os
# Current work
print(os.getcwd())
# Change directory 2 level up
os.chdir("..")
os.chdir("..")
os.chdir("..")

print(os.getcwd())
# Look for the file in the directory
print(os.listdir())

file_path = 'experiments/experiment_2/data/13428_2013_403_MOESM1_ESM.xlsx'

try:
    concreteness_data = pd.read_excel(file_path, sheet_name=0)
    
    preview = concreteness_data.head()
except Exception as e:
    preview = str(e)

preview


/tmp/ipykernel_30792/75514973.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


/home/jose/Git/SE-AP/experiments/experiment_2/notebooks
/home/jose/Git/SE-AP
['models', 'todo.md', '.pre-commit-config.yaml', 'demos', '.gitignore', 'experiments', 'include', '.git', '.env', 'README.md', '.env-example', 'LICENSE', 'model_technique_pathology_scores.csv']


,Word,Bigram,Conc.M,Conc.SD,Unknown,Total,Percent_known,SUBTLEX
0,a,0,1.46,1.14,2,30,0.933333,1041179
1,aardvark,0,4.68,0.86,0,28,1.000000,21
2,aback,0,1.65,1.07,4,27,0.851852,15
3,abacus,0,4.52,1.12,2,29,0.931034,12
4,abandon,0,2.54,1.45,1,27,0.962963,413


## looking and processing data

In [2]:
from experiments.experiment_2.data import words

MWB1=words.MATERIAL_WELL_BEING['Concrete']
MWB1 = [word if word != "books" else "book" for word in MWB1] # fix at the end replacing book with books
MWB2=words.MATERIAL_WELL_BEING['Abstract']

H1=words.HEALTH['Concrete']
H1 = [word if word != "cigarettes" else "cigarette" for word in H1] # fix at the end replacing cigarette with cigarettes
H1 = [word if word != "fruits" else "fruit" for word in H1] # fix at the end replacing fruit with fruits

H2=words.HEALTH['Abstract']

P1=words.PRODUCTIVITY['Concrete']
P2=words.PRODUCTIVITY['Abstract']
print(P1)
P1= [word if word != "time-management" else "timekeeping" for word in P1] # fix at the end replacing timekeeping with time-management
print(P1)
I1=words.INTIMACY['Concrete']
I1= [word if word != "bickering" else "bicker" for word in I1] # fix at the end replacing bickering with squabbling
I2=words.INTIMACY['Abstract']
I1= [word if word != "sharing" else "share" for word in I1] # fix at the end replacing bickering with squabbling
print(I1)

ALL_LIST = MWB1 + MWB2 + H1 + H2 + P1 + P2 + I1 + I2
#imprimir len de all list
print(len(ALL_LIST))
ALL_LIST_CONCRETE = words.MATERIAL_WELL_BEING['Concrete'] + words.HEALTH['Concrete'] + words.PRODUCTIVITY['Concrete'] + words.INTIMACY['Concrete']
ALL_LIST_ABSTRACT = words.MATERIAL_WELL_BEING['Abstract'] + words.HEALTH['Abstract'] + words.PRODUCTIVITY['Abstract'] + words.INTIMACY['Abstract']

['alarm', 'boss', 'calendar', 'chat', 'clock', 'contract', 'email', 'employee', 'factory', 'game', 'learning', 'meeting', 'noise', 'office', 'organization', 'party', 'presentation', 'report', 'salary', 'time-management']
['alarm', 'boss', 'calendar', 'chat', 'clock', 'contract', 'email', 'employee', 'factory', 'game', 'learning', 'meeting', 'noise', 'office', 'organization', 'party', 'presentation', 'report', 'salary', 'timekeeping']
['bicker', 'caress', 'coldness', 'compliment', 'connection', 'conversation', 'cuddle', 'date', 'dinner', 'distance', 'divorce', 'frown', 'gift', 'handshake', 'hit', 'hug', 'kiss', 'sex', 'share', 'smile']
160


### ALL LIST

In [3]:
filtered_data = concreteness_data[concreteness_data['Word'].isin(ALL_LIST)]
filtered_data = filtered_data.sort_values('Word')

missing_words = set(ALL_LIST) - set(filtered_data['Word'])

filtered_data, missing_words

(                 Word  Bigram  Conc.M  Conc.SD  Unknown  Total  Percent_known  \
 248       achievement       0    2.37     1.50        0     30            1.0   
 408        admiration       0    1.86     1.06        0     29            1.0   
 622            agenda       0    2.66     1.32        0     29            1.0   
 638    aggressiveness       0    2.59     1.19        0     27            1.0   
 681               aid       0    3.10     1.47        0     29            1.0   
 ...               ...     ...     ...      ...      ...    ...            ...   
 36033           watch       0    4.61     0.83        0     28            1.0   
 36047           water       0    5.00     0.00        0     28            1.0   
 36117        weakness       0    2.59     1.47        0     27            1.0   
 36118          wealth       0    2.63     1.50        0     27            1.0   
 36911           yacht       0    4.97     0.19        0     29            1.0   
 
        SUBTLE

## Median-split

In [4]:
median_concreteness = filtered_data['Conc.M'].median()

filtered_data["ConcretenessCategory"] = filtered_data["Conc.M"].apply(lambda x: "Concrete" if x >= median_concreteness else "Abstract")

filtered_data['Word'] = filtered_data['Word'].apply(lambda x: 'books' if x == 'book' else x)
filtered_data['Word'] = filtered_data['Word'].apply(lambda x: 'cigarettes' if x == 'cigarette' else x)
filtered_data['Word'] = filtered_data['Word'].apply(lambda x: 'fruits' if x == 'fruit' else x)
filtered_data['Word'] = filtered_data['Word'].apply(lambda x: 'time-management' if x == 'timekeeping' else x)
filtered_data['Word'] = filtered_data['Word'].apply(lambda x: 'bickering' if x == 'bicker' else x)
filtered_data['Word'] = filtered_data['Word'].apply(lambda x: 'sharing' if x == 'share' else x)

concrete_words = filtered_data[filtered_data['ConcretenessCategory'] == 'Concrete']['Word'].head(80).tolist()
abstract_words = filtered_data[filtered_data['ConcretenessCategory'] == 'Abstract']['Word'].head(80).tolist()

dict_r = {
    'concrete': concrete_words,
    'abstract': abstract_words
}


median_concreteness, filtered_data[['Word', 'Conc.M', 'ConcretenessCategory']].sort_values(by='Conc.M')

(3.125,
              Word  Conc.M ConcretenessCategory
 16872  insecurity    1.52             Abstract
 19027     loyalty    1.57             Abstract
 15663   ignorance    1.60             Abstract
 23104       peace    1.62             Abstract
 35445       value    1.62             Abstract
 ...           ...     ...                  ...
 36911       yacht    4.97             Concrete
 5290        clock    5.00             Concrete
 36047       water    5.00             Concrete
 15255       house    5.00             Concrete
 4165       camera    5.00             Concrete
 
 [160 rows x 3 columns])

In [5]:
dict_r
#imprimir el len de cada categoria
print(len(concrete_words))
print(len(abstract_words))

80
80


## Comparison

### MWB

In [6]:
all_set_concrete = set(ALL_LIST_CONCRETE)
all_set_abstract = set(ALL_LIST_ABSTRACT)

result_all_set_concrete = set(dict_r['concrete'])
result_all_set_abstract = set(dict_r['abstract'])

diferencias_concrete = all_set_concrete.difference(result_all_set_concrete)
diferencias_abstract = all_set_abstract.difference(result_all_set_abstract)

print("Words in the original list that are not in the result list (concrete):", diferencias_concrete)
print("Words in the original list that are not in the result list (abstract):", diferencias_abstract)

Words in the original list that are not in the result list (concrete): {'connection', 'compliment', 'time-management', 'sharing', 'presentation', 'organization'}
Words in the original list that are not in the result list (abstract): {'savings', 'protein', 'illness', 'hazard', 'diet', 'property'}


## Save data

In [7]:
import json

with open('experiments/experiment_2/data/concreteness.json', 'w') as f:
    json.dump(dict_r, f, indent=4)